In [1]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

sns.set()


 Derivative-Free Optimization for Least-Squares (DFO-LS) is a
 nonlinear least-squares solver that only requires function values.

 Documentation
 [https://numericalalgorithmsgroup.github.io/dfols/build/html/index.html](https://numericalalgorithmsgroup.github.io/dfols/build/html/index.html)

 Repository
 [https://github.com/numericalalgorithmsgroup/dfols](https://github.com/numericalalgorithmsgroup/dfols)

 Install using either:
 `conda install -c conda-forge dfo-ls`
 `pip install DFO-LS`

In [2]:
from dfols import solve


In [3]:
# project files
from rp_model.files import set_files_directory
set_files_directory("../files")

from rp_model.utils import pack, table, DataStore
from rp_model.calc import FitOptions, compute_rp, make_precomputed_columns, make_initial_guess

RP model file path: C:\Users\jckin\Documents\Untitled Folder\RP fit\files


In [4]:
# stuff for display

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# progress bar

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 120)

In [5]:
data = pd.read_pickle(FitOptions.data_file)
data.describe()
data.head()

,Level,RP,MS lvl,ModelRP,Difference,Freq1,Berry1,Ing1P,NrgNat,Helps per hour,IngrMult,SkillMult,BerryD,IngD,SklVal,FreqL,HelpNat,IngrNat,Ingr%,SkillNat,Skill%,BerryL,Dupes,Diff -0.1% ingr,Diff +0.1% ingr,Diff,Amnt,Ing2P,MiscMult,RP Multiplier,HelpSub,IngrSub,SkillSub
count,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,11541.0,11541.000000,11541.000000,11541.000000,11541.000000,11541.000000,7940.000000,11541.000000,7940.000000,7940.000000,7940.000000
mean,14.839269,797.265575,1.674898,797.266008,0.000433,3756.164977,28.149554,108.637380,0.993491,1.102978,0.210948,0.039065,64.076856,154.928689,980.406291,3565.881629,0.987549,0.999307,0.203650,1.009219,0.036717,42.636513,1.0,-0.677398,0.602912,-0.037119,0.252924,12.261589,1.054267,1.043974,0.973472,1.046428,1.060438
std,9.945906,615.837107,1.154834,615.837077,0.020811,1009.463620,2.748497,15.988426,0.043885,0.345022,0.058564,0.022347,37.954587,70.354132,903.623248,1075.090716,0.057112,0.117452,0.040759,0.120275,0.019606,12.343025,0.0,1.252369,1.249570,1.013367,0.874277,39.561162,0.103810,0.085322,0.051903,0.113089,0.126464
min,1.000000,245.000000,1.000000,245.000000,0.000000,2100.000000,24.000000,90.000000,0.920000,0.520000,0.100000,0.006400,25.000000,90.000000,400.000000,1407.780000,0.900000,0.800000,0.130000,0.800000,0.010000,24.000000,1.0,-57.930000,-58.750000,-58.340000,0.000000,0.000000,0.920000,1.000000,0.790000,1.000000,1.000000
25%,8.000000,394.000000,1.000000,394.000000,0.000000,2800.000000,26.000000,98.000000,1.000000,0.820000,0.170000,0.020000,39.000000,101.000000,400.000000,2656.800000,1.000000,1.000000,0.170000,1.000000,0.020000,34.000000,1.0,-0.910000,0.140000,-0.250000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,12.000000,584.000000,1.000000,584.000000,0.000000,3700.000000,28.000000,103.000000,1.000000,1.050000,0.202400,0.036000,52.000000,121.000000,880.000000,3427.200000,1.000000,1.000000,0.200000,1.000000,0.040000,40.000000,1.0,-0.506000,0.490000,-0.001600,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,20.000000,929.000000,2.000000,929.000000,0.000000,4400.000000,31.000000,115.000000,1.000000,1.350000,0.247900,0.050400,76.000000,206.000000,1083.000000,4347.200000,1.000000,1.000000,0.230000,1.000000,0.050000,48.000000,1.0,-0.160000,0.920000,0.240000,0.000000,0.000000,1.120000,1.000000,1.000000,1.000000,1.000000
max,55.000000,5338.000000,7.000000,5338.000000,1.000000,6300.000000,35.000000,151.000000,1.080000,2.550000,0.517400,0.220000,351.000000,511.000000,12438.000000,6916.140000,1.100000,1.200000,0.340000,1.200000,0.140000,117.000000,1.0,0.950000,7.840000,0.510000,8.000000,342.000000,1.550000,1.440000,1.000000,1.540000,1.540000


,Pokemon,Level,RP,NatureEN/ZH,MS lvl,Source,ModelRP,Difference,Nature,ID,NatureP,NatureN,Type,Class,MSkill,Freq1,Berry1,Ing1,Ing1P,NrgNat,Helps per hour,IngrMult,SkillMult,BerryD,IngD,SklVal,FreqL,HelpNat,IngrNat,Ingr%,SkillNat,Skill%,BerryL,Dupes,DupeMatch,Diff -0.1% ingr,Diff +0.1% ingr,Diff,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,MiscMult,RP Multiplier,HelpSub,IngrSub,SkillSub,Sub Skill 3
0,Arbok,8.0,531.0,Hardy,1.0,Questions help guides,531.0,0.0,Neutral,ArbokNeutral81,-,-,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,0.98,0.26,0.06,78.0,103.0,400.0,3648.20,1.0,1.0,0.26,1.0,0.06,39,1,True,-0.4900,-0.2200,-0.3600,,,,0.0,0.0,NaN,1.0,NaN,NaN,NaN,
1,Arbok,8.0,538.0,Quiet,1.0,Questions help guides,538.0,0.0,Quiet,ArbokQuiet81,Ingredient Finding,Exp Gains,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,0.98,0.32,0.06,78.0,103.0,400.0,3648.20,1.0,1.2,0.26,1.0,0.06,39,1,True,-0.1400,0.1300,-0.0100,,,,0.0,0.0,NaN,1.0,NaN,NaN,NaN,
2,Arbok,9.0,528.0,Bold,1.0,,528.0,0.0,Bold,ArbokBold91,Energy Recovery,Speed of Help,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.08,0.89,0.26,0.06,80.0,103.0,400.0,4004.88,1.1,1.0,0.26,1.0,0.06,40,1,True,0.1956,0.4548,0.3252,,,,0.0,0.0,NaN,1.0,NaN,NaN,NaN,
3,Arbok,9.0,539.0,Hardy,1.0,Double Jumper,539.0,0.0,Neutral,ArbokNeutral91,-,-,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,0.98,0.26,0.06,80.0,103.0,400.0,3640.80,1.0,1.0,0.26,1.0,0.06,40,1,True,-0.4700,-0.2000,-0.3400,,,,0.0,0.0,NaN,1.0,NaN,NaN,NaN,
4,Arbok,9.0,574.0,Naughty,1.0,Rate My Mon,574.0,0.0,Naughty,ArbokNaughty91,Speed of Help,Main Skill Chance,Poison,Berries,Charge Energy S,3700.0,32.0,Bean Sausage,103.0,1.00,1.09,0.26,0.05,80.0,103.0,400.0,3276.72,0.9,1.0,0.26,0.8,0.06,40,1,True,0.1200,0.4100,0.2600,,,,0.0,0.0,NaN,1.0,NaN,NaN,NaN,


In [7]:
# Make an initial guess
# Pack the dictionary into a 1-D vector.
# Also store information on how to unpack that vector

x0, unpack_info = pack(*make_initial_guess())
computed = make_precomputed_columns(data)

In [8]:
# We compute the RP model in rp_model.py
# Here we create closure to glue the pieces together.

def RP(x):
    return compute_rp(x, data, computed, unpack_info)


# for the error analysis page,
# we'll compute the residual with exact rounding.

def residual(x):
    return data["RP"] - np.round(RP(x))


In [9]:
store = (DataStore(FitOptions.result_file)
         .with_dependency_on(data, x0)
         .try_read_and_validate()
         )

opt = store.data()

AttributeError: type object 'FitOptions' has no attribute 'get_result_file'

In [ ]:

#FitOptions.soft_round.alpha = 18

optx = opt.x
result = solve(residual, optx, print_progress=True, maxfun=2200)


In [ ]:
table(result)

In [ ]:
r_opt = data["RP"] - np.round(RP(result.x))
pd.Series(r_opt).describe()

_ = plt.figure()
g = sns.histplot(x=np.clip(np.abs(r_opt), 1e-1, 1e3), log_scale=True, bins=20)
g.axes.set_yscale('log')
#_ = g.bar_label(g.containers[1], labels=[str(v) if v else '' for v in g.containers[1].datavalues])

_ = plt.figure()
g = sns.histplot(x=r_opt, discrete=1)
_ = g.axes.set_yscale('log')
#_ = g.bar_label(g.containers[1], labels=[str(v) if v else '' for v in g.containers[1].datavalues])

In [ ]:

# What are the worst points?

df = data[["Pokemon", "ID", "Level", "MS lvl", "NatureP", "NatureN", "Sub Skill 1", "Sub Skill 2", "RP"]].copy()

df["fit"] = RP(result.x)
df["error"] = data["RP"] - np.round(RP(result.x))
df["error2"] = df["error"] ** 2

df = df.sort_values("error2", ascending=False).reset_index(drop=True)
df.head(100)
